## LP - Summation

In [7]:
import pyomo.environ as pyo
from pyomo.environ import *
from pyomo.opt import SolverFactory
import pandas as pd

In [29]:
# inputs
dataGen = pd.read_excel("inputs.xlsx", sheet_name="gen")
dataLoad = pd.read_excel("inputs.xlsx", sheet_name="load")
Ng = len(dataGen)
Nd = len(dataLoad)

In [51]:
#model
model = pyo.ConcreteModel()

model.Pg = pyo.Var(range(Ng), bounds=(0,None))
Pg = model.Pg

# model.Pd = pyo.Var(range(Nd), bounds=(0,None))
# Pd = model.Pd

#constraints
pg_sum =  sum([Pg[g] for g in dataGen.id])
pd_sum =  sum([Pd[d] for d in dataLoad.id])
model.balance = pyo.Constraint(expr = pg_sum == sum(dataLoad.value))

model.cond = pyo.Constraint(expr = Pg[0]+Pg[3] >= dataLoad.value[0])

model.limits = pyo.ConstraintList()
for g in dataGen.id:
    model.limits.add(expr= Pg[g] <= dataGen.limit[g])

#objFun
cost_sum = sum([Pg[g]*dataGen.cost[g] for g in dataGen.id])
model.obj = pyo.Objective(expr = cost_sum)

opt = SolverFactory('glpk')
results = opt.solve(model)

dataGen['Pg'] = [pyo.value(Pg[g]) for g in dataGen.id]
print(dataGen)

   id  limit  cost    Pg
0   0     20  0.10  20.0
1   1     10  0.05  10.0
2   2     40  0.30  35.0
3   3     50  0.40  30.0
4   4      5  0.01   5.0


In [49]:
print(sum([Pg[g]*dataGen.cost[g] for g in dataGen.id]))

0.1*Pg[0] + 0.05*Pg[1] + 0.3*Pg[2] + 0.4*Pg[3] + 0.01*Pg[4]


In [47]:
model.pprint()

2 Set Declarations
    Pg_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    5 : {0, 1, 2, 3, 4}
    limits_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    5 : {1, 2, 3, 4, 5}

1 Var Declarations
    Pg : Size=5, Index=Pg_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :  None :  None : False :  True :  Reals
          1 :     0 :  None :  None : False :  True :  Reals
          2 :     0 :  None :  None : False :  True :  Reals
          3 :     0 :  None :  None : False :  True :  Reals
          4 :     0 :  None :  None : False :  True :  Reals

3 Constraint Declarations
    balance : Size=1, Index=None, Active=True
        Key  : Lower : Body                                  : Upper : Active
        None : 100.0 : Pg[0] + Pg[1] + Pg[2] + Pg[3] + Pg[4] : 100.0 :   True
    cond : Size=1, I